In [ ]:
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/Colab_Notebooks/praknlp'
drive.mount('/content/gdrive', force_remount=True)
os.chdir(gdrive_path)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import spacy
import nltk
import pickle
import torch
import re
import json

import tensorflow_hub as hub
import tensorflow as tf
from nltk import word_tokenize
nltk.download('punkt')
import torch.nn.functional as F

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open('./ELMo_model/ingredient_embed.pkl', 'rb') as fil:
  ingredient_embed_dict = pickle.load(fil)
with open('./ELMo_model/ingredient_freq.pkl', 'rb') as f:
  ingredient_freq = pickle.load(f)

In [ ]:
print(ingredient_freq)

{'butter': 104, 'oil': 97, 'penne': 1, 'pasta': 36, 'water': 87, 'sauce': 76, 'sprinkle': 56, 'chili': 7, 'cheese': 94, 'flour': 59, 'milk': 41, 'salt': 113, 'garlic': 46, 'chipotle': 1, 'liquid': 12, 'macaroni': 5, 'celery': 12, 'green': 18, 'pepper': 89, 'pimento': 1, 'mayonnaise': 14, 'salad': 17, 'dressing': 15, 'vinegar': 14, 'dill': 3, 'lettuce': 3, 'tomato': 10, 'olive': 18, 'garnish': 16, 'basil': 10, 'corn': 18, 'potato': 4, 'jello': 2, 'watermelon': 2, 'crust': 20, 'toast': 6, 'coconut': 10, 'crisp': 8, 'beef': 14, 'lemon': 37, 'juice': 41, 'soy': 12, 'cornstarch': 8, 'pineapple': 5, 'orange': 11, 'rice': 16, 'cashew': 1, 'chicken': 79, 'tea': 2, 'meat': 28, 'sugar': 95, 'vanilla': 22, 'zucchini': 14, 'loaf': 11, 'red': 14, 'salmon': 10, 'fillet': 2, 'ginger': 10, 'black': 12, 'fish': 8, 'la': 1, 'cookie': 15, 'parmesan': 11, 'topping': 8, 'banana': 2, 'kiwi': 1, 'cinnamon': 19, 'yogurt': 4, 'fruit': 9, 'fennel': 9, 'pork': 10, 'wine': 14, 'broth': 20, 'roast': 12, 'hot': 23,

In [ ]:
with open('./ELMo_embeddings/used_ingredients.json','r') as fp:
  used_ing = json.load(fp)
print(len(used_ing))

4418


In [ ]:
if 'tomato' in ingredient_freq:
  print(ingredient_freq['tomato'])
else: print('no')

10


In [ ]:
recipe = 'Take a glass of milk and add cheese to it' #test sentence for validation/prediction to be entered here

recipe = re.sub(r'\.\ \)\.','\)\.',recipe)
recipe = re.sub(r'[^\.\-\!\?\w\s]','',recipe)
recipe = re.sub(r'[^\.\-\w\s]','.',recipe)
recipe = re.sub(r'[^\.\w\s]',' ',recipe)
recipe = re.sub(r'\d+', '', recipe)
recipe = recipe.lower()
print(recipe)

words = word_tokenize(recipe)

items = []
for j in words:
  if j in used_ing and j not in items:
    items.append(j)

for item in items:
  print('Ingredient observed: ' + str(item))
  x = ingredient_embed_dict[item]/ingredient_freq[item]
  print(ingredient_embed_dict[item])
  print(ingredient_freq[item])
  print(x)
  print('Possible alternative:')
  sim = 1
  for k,v in ingredient_embed_dict:
    cos = F.cosine_similarity(torch.Tensor(ingredient_embed_dict[item]), torch.Tensor(v), dim=0)
    if cos < sim:
      ideal = k
      sim = cos
  print(ideal)

take a glass of milk and add cheese to it
Ingredient observed:milk
[ 10.66022   10.715646  21.267382 ...   7.569105 -12.836133  21.115639]
41
[ 0.26000535  0.26135722  0.51871663 ...  0.18461232 -0.3130764
  0.5150156 ]
Possible alternatives:
Ingredient observed:cheese
[ 27.61032   20.60123   51.690792 ...   8.280687 -34.90123   40.131474]
94
[ 0.2937268   0.21916203  0.549902   ...  0.08809242 -0.37128967
  0.42693058]
Possible alternatives:
